In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
from matplotlib import pylab as plt

import time
import rascal

import ase
from ase import Atoms
from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view
import numpy as np

# Descriptor related imports: compare the librascal and pyLODE versions of SOAP
import rascal.representations
import rascaline
from pylode.lib.projection_coeffs import DensityProjectionCalculator

# Introduction

This notebook is used to compare the features obtained from rascaline, librascal and pylode. The goal is to fully understand the details of the three implementations to make sure that the same coefficients from different codes agree.

# Compute the same features using 3 different libraries

### Define structure to be used for the comparison
The main differences between librascal and the pyLODE implementation are the presence of a smooth cutoff function and the potentially different order in which neighbors are stored. We thus wish to generate structures for which these two effects do not alter the coefficients too much. For this task, we use clusters of Oxygen atoms for which all atoms have a mutual distance of less than 3A. Then, even for a relatively large smearing of 1.5A, the atomic densities will be reasonably contained within a ball of cutoff radius 6 that will also be used for the cutoff.

In [ ]:
frames = []
cell = np.eye(3) * 16
distances = np.linspace(1., 2.5, 10)
for d in distances:
    positions = [[1,1,1],[1+0.1*d,1+d,d+1]]
    positions = [[1,1,1],[1,1,1+d]]
    frame = Atoms('O2', positions=positions, cell=cell, pbc=True)
    frames.append(frame)

### Define common hyperparameters

In [ ]:
nmax = 5
lmax = 2
rcut = 6.
smearing = 0.5

### Get the features from librascal

In [ ]:
# define the parameters of the spherical expansion
hypers = dict(interaction_cutoff=rcut, 
              max_radial=nmax, 
              max_angular=lmax, 
              gaussian_sigma_constant=smearing,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.1,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )

calculator_librascal = rascal.representations.SphericalExpansion(**hypers)
# compute the representation of all the structures
features_librascal = calculator_librascal.transform(frames).get_features(calculator_librascal)

### Get the features from pyLODE

In [ ]:
hypers = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':nmax,
    'cutoff_radius':rcut,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False,
    }

calculator_pylode = DensityProjectionCalculator(**hypers)
calculator_pylode.transform(frames)
features_pylode = calculator_pylode.features

### Get the features from Rascaline

In [ ]:
HYPER_PARAMETERS = {
    "cutoff": rcut,
    "max_radial": nmax,
    "max_angular": lmax,
    "atomic_gaussian_width": smearing,
    "gradients": False,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "Step": {"width": 1e-3},
    },
}

calculator_rascaline = rascaline.SphericalExpansion(**HYPER_PARAMETERS)

# run the actual calculation
descriptor_rascaline = calculator_rascaline.compute(frames)

In [ ]:
print(descriptor_rascaline.values[0].reshape(((lmax+1)**2, nmax)).T / 10.962374348347298)

In [ ]:
print(features_librascal[0].reshape((nmax,(lmax+1)**2)))

In [ ]:
print(np.round(features_pylode,13)[0,0])

In [ ]:
10.962374348347298 / np.sqrt(2)

# Debugging: Compare coeffs with semi-analytical results

### Generate orthonormalization matrix

In [ ]:
from scipy.integrate import quad
from pylode.lib.radial_basis import innerprod

In [ ]:
Nradial = 1000
sigma = np.ones(nmax, dtype=float)
for i in range(1, nmax):
    sigma[i] = np.sqrt(i)
sigma *= rcut / nmax

# Define primitive GTO-like radial basis functions
f_gto = lambda n, x: x**n * np.exp(-0.5 * (x / sigma[n])**2)
xx = np.linspace(0, rcut * 2.5, Nradial)
R_n = np.array([f_gto(n, xx) for n in range(nmax)])

# Orthonormalize
innerprods = np.zeros((nmax, nmax))
for i in range(nmax):
    for j in range(nmax):
        innerprods[i, j] = innerprod(xx, R_n[i], R_n[j])
eigvals, eigvecs = np.linalg.eigh(innerprods)
transformation = eigvecs @ np.diag(np.sqrt(1. / eigvals)) @ eigvecs.T

### Compute the coefficients for $l=1$ starting from the analytical formula

In [ ]:
def compute_coefficient(dist = 2.0):
    prefac = np.sqrt(1.5) / (2 * np.pi * smearing**3)
    prefac *= np.exp(-0.5 * d**2 / smearing**2)
    
    # Define sigma_n for each radial basis function
    sigma_radial = np.ones(nmax, dtype=float)
    for i in range(1,nmax):
        sigma_radial[i] = np.sqrt(i)
    sigma_radial *= rcut/nmax

    # Compute coefficients
    coeffs = np.zeros((nmax,))
    for n in range(nmax):
        # Start defining functions appearing in integrand
        R_n = lambda r: r**n * np.exp(-0.5*r**2/smearing**2)
        gaussian = lambda r: np.exp(-0.5*r**2/sigma_radial[n]**2)
        reff = lambda r: d*r/smearing**2
        hyperbolic = lambda r: 2*(reff(r)*np.cosh(reff(r)) + np.sinh(reff(r)))/reff(r)**2
        integrand = lambda r: r**2 * R_n(r) * gaussian(r) * hyperbolic(r)
        
        # Output
        eps = 1e-8
        coeffs[n] = prefac * quad(integrand, eps, 20)[0]
    
    return coeffs

In [ ]:
features_explicit = np.zeros((len(distances), nmax))
for i, d in enumerate(distances):
    features_explicit[i] = transformation @ compute_coefficient(dist = d)

In [ ]:
print(features_explicit)

### Get the $l=1$ part of the features of the three codes

In [ ]:
print('Shapes of feature vectors')
print('rascaline', descriptor_rascaline.values.shape)
print('librascal', features_librascal.shape)

In [ ]:
features_L1_rascaline = descriptor_rascaline.values[0::2, 2*nmax:3*nmax] / 10.962374348347298 

In [ ]:
features_L1_librascal = features_librascal[0::2, 2::(lmax+1)**2]

In [ ]:
print(np.round(features_L1_rascaline, 6))

In [ ]:
ratios = np.round(features_L1_rascaline / features_explicit, 5)
print(ratios)

In [ ]:
hypers_pylode_primitive = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':nmax,
    'cutoff_radius':rcut,
    'potential_exponent':0,
    'radial_basis': 'gto_primitive',
    'compute_gradients':False,
    }

calculator_pylode_primitive = DensityProjectionCalculator(**hypers_pylode_primitive)
calculator_pylode_primitive.transform(frames)
features_pylode_primitive = np.round(calculator_pylode_primitive.features, 15)

In [ ]:
features_pylode_primitive.shape

In [ ]:
features_pylode_primitive[0::2,0,:,2]

In [ ]:
features_explicit

In [ ]:
features_explicit / features_pylode_primitive[1::2,0,:,2]